In [128]:
import os
import sys
import random
import numpy as np

Load MNIST data

In [63]:
import pickle 
import gzip
f = gzip.open('mnist.pkl.gz', 'rb')
training, validation, test = pickle.load(f, encoding='latin1')
f.close()

In [81]:
images = training[0][:] 
labels = training[1][:]

50000


In [348]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))
def dsigmoid(x):
    return x * (1.0 - x)
def softplus(x):
    return np.log(1.0 + np.exp(x))
def dsoftplus(x):
    return 1.0 - np.exp(-x)
def softmax(x):
    ex = np.exp(x)
    return ex / np.sum(ex)
def dsoftmax(x):
    return x*(1.0-x)

In [354]:
nneurons = 500
noutput  = 10
nintlayers = 3
initialscale = 0.1
class Layer():
    def __init__(self, nneurons, afunc):
        self.nneurons=nneurons
        self.afunc = afunc
        if afunc == sigmoid:
            self.dfunc = dsigmoid
        if afunc == softplus:
            self.dfunc = dsoftplus
        if afunc == softmax:
            self.dfunc = dsoftmax
        self.b = np.random.normal(size=nneurons) * initialscale
        self.db = np.zeros(nneurons)
        

In [355]:
nnetwork = [Layer(len(images[1]), sigmoid)]
weigth = []
dw = []
for i in range(nintlayers):
    nnetwork.append(Layer(nneurons, softplus))
    weigth.append(np.random.normal( \
        size=[nnetwork[i].nneurons, nneurons]) * initialscale)
    dw.append(np.zeros([nnetwork[i].nneurons, nneurons]))
nnetwork.append(Layer(noutput, softmax))
weigth.append(np.random.normal(     \
        size=[nnetwork[nintlayers].nneurons, noutput]) * initialscale)
dw.append(np.zeros([nnetwork[nintlayers].nneurons, noutput]))

In [356]:
learningrate = 0.001
dropout      = 0.5
momentum     = 0.75
nbatch       = 100
nimages      = len(images)
nupdates     = round(nimages / nbatch) * 100

def remove(n):
    return np.where(np.random.random(n) > dropout, 1, 0)

In [357]:
nupdates = 1
for i in range(nupdates):
    
    r = [ remove(l.nneurons) for l in nnetwork[1:-1] ] 

    batchindexes = np.random.randint(nimages, size=nbatch)
    x = 6.0 * images[batchindexes] + 3
    expected = np.zeros([nbatch, noutput])
    for i, j in enumerate(batchindexes):
        expected[i, labels[j]] = 1

        
    y = []
    y.append(nnetwork[0].afunc(x + nnetwork[0].b))
    for i in range(1, nintlayers+2):
        # transpose of product is product of transposes in reverse order
        y.append(nnetwork[i].afunc( 
            np.matmul(y[i-1], weigth[i-1]) + nnetwork[i].b ))

        
    e = [[] for i in range(nintlayers + 2)]
    e[nintlayers + 1] = expected - y[nintlayers +1]
    for i in range(nintlayers, 0, -1):
        print(i)
        e[i] = np.dot( 
            np.matmul(e[i+1], weigth[i].transpose()).transpose(),
            nnetwork[i].dfunc(y[i])
              )
    
    

    

3
2


ValueError: shapes (500,500) and (100,500) not aligned: 500 (dim 1) != 100 (dim 0)